In [1]:
import torch
# model_silero_te, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
#                                                                   model='silero_te')
import re
import string
from nltk.tokenize import word_tokenize,TreebankWordTokenizer


In [2]:
import soundfile

# load model and processor
import os

# import librosa
import jiwer
import string
import csv
from autocorrect import Speller

spell = Speller()
from transformers import pipeline

# fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")


def get_gold_hyp_lists(gold_path, test_path, septype):
    def create_hyp_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, "r") as file:
            lines = file.readlines()
            for idx, line in enumerate(lines):
                line = line.strip()
                if line and len(line.split(septype, 1)) == 2:
                    # print(line.split(septype, 1))
                    sentence_id, sentence = line.split(septype, 1)
                    # sentence_id=sentence_id+".wav" # for AMI
                    sentence_id = sentence_id.split(".", 1)[0]  # cv
                    sentence_dict[sentence_id] = sentence
        return sentence_dict

    def create_gold_sentence_dictionary(filename):
        sentence_dict = {}
        with open(filename, "r") as file:
            for line in file:
                # parts = line.split(' ', 1) # Libri
                parts = line.split(",", 1)  # AMI
                sentence_id, sentence = parts
                # print(sentence_id)
                # sentence_id = sentence_id.split("/", 1)[1]  # AMI
                sentence_id = sentence_id.split(".", 1)[0]  # cv
                # print(sentence_id)
                # print(sentence.strip())
                if sentence.strip() != "":
                    sentence_dict[sentence_id] = sentence.strip()
        return sentence_dict

    def read_csv_to_dict(file_path):
        result_dict = {}
        with open(file_path, "r") as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
                key, value = row[0], row[1]
                key = key.split("/")[1]
                # print(key,value)
                # break
                result_dict[key] = value
        return result_dict

    gold_dict = create_gold_sentence_dictionary(gold_path)
    # gold_dict = read_csv_to_dict(gold_path)
    test_dict = create_hyp_sentence_dictionary(test_path)

    gold_list = []
    test_list = []
    for test_key, test_value in test_dict.items():
        # print(test_key)
        if test_key in gold_dict:
            gold = gold_dict[test_key]
            gold_list.append(gold)
            test_list.append(test_value)
    print(len(test_list))
    return gold_list, test_list


def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ""
    for char in text:
        if char in punctuation_marks:
            modified_text += " " + char
        else:
            modified_text += char

    return modified_text


def add_space_before_punctuation(text):
    # List of punctuation marks
    punctuation_marks = string.punctuation

    # Iterate over each character in the text
    modified_text = ""
    for char in text:
        if char in punctuation_marks:
            modified_text += " " + char
        else:
            modified_text += char

    return modified_text


def add_space(reference_list, hypothesis_list):
    reference = []
    hypothesis = []
    for ref, hyp in zip(reference_list, hypothesis_list):
        # Preprocess sentences to keep punctuation as separate words
        reference.append(add_space_before_punctuation(ref))
        hypothesis.append(add_space_before_punctuation(hyp))
        # print(reference,hypothesis)
        # Calculate WER

    #     wer = jiwer.wer(reference, hypothesis)
    #     # print(wer)
    #     wer_scores.append(wer)
    # wer_mean=sum(wer_scores) / len(wer_scores)

    return reference, hypothesis


def get_detail_wer(gold_list, test_list):
    out = jiwer.process_words(
        gold_list,
        test_list,
    )
    print("wer: ", out.wer * 100)
    print("hits: ", out.hits)
    print("substitutions: ", out.substitutions)
    print("insertions: ", out.insertions)
    print("deletions: ", out.deletions)
    print("total words: ", out.hits + out.substitutions + out.deletions)


# def get_t_p_wer(substitutions, insertions,deletions,total_words_punc, P_WER):
#     t_p_wer=(substitutions+insertions+deletions)/total_words_punc
#     print("Topline of P-WER is ",t_p_wer*100)
#     print("Relative PER is ", P_WER-t_p_wer*100)


def get_c_p_er_fr_wer(wer, C_WER, P_WER, clean=True):
    if clean:
        total_words = 50082
        total_words_punt = 57312
        cased_words = 3849
        p_num = 9709
    else:
        total_words = 48488
        total_words_punt = 55896
        cased_words = 4231
        p_num = 9408
    error_words = wer * total_words
    error_c = C_WER * total_words
    cer = (error_c - error_words) / cased_words
    error_p = P_WER * total_words_punt
    per = (error_p - error_words) / p_num
    # print("Topline of P-WER is ",t_p_wer)
    print("Relative CER is ", cer)
    print("Relative PER is ", per)


def get_c_p_er_fr_wer_ami(wer, C_WER, P_WER):
    # if clean:
    total_words = 94006
    total_words_punt = 110266
    cased_words = 7626
    p_num = 16260
    error_words = wer * total_words
    error_c = C_WER * total_words
    cer = (error_c - error_words) / cased_words
    error_p = P_WER * total_words_punt
    per = (error_p - error_words) / p_num
    print("Relative CER is ", cer)
    print("Relative PER is ", per)


def get_c_p_er_fr_wer_cv(wer, C_WER, P_WER):
    # if clean:
    total_words = 11244
    total_words_punt = 13146
    cased_words = 1882
    p_num = 1902
    error_words = wer * total_words
    error_c = C_WER * total_words
    cer = (error_c - error_words) / cased_words
    error_p = P_WER * total_words_punt
    per = (error_p - error_words) / p_num
    print("Relative CER is ", cer)
    print("Relative PER is ", per)


def get_cased_word(text_list):
    capitalized_word_count = 0

    # Iterate through the list of text
    for text in text_list:
        # Split the text into words
        words = text.split()

        # Check each word in the text
        for word in words:
            # Check if the word is capitalized (first character is uppercase)
            if word.isalpha() and word[0].isupper():
                capitalized_word_count += 1
    print("Cased words number is ", capitalized_word_count)

In [3]:
gold_list, test_list = get_gold_hyp_lists(
    "/Users/ccui/Desktop/POC/dictation/CommonVoice/cv-en_test-2k_len4-10.txt",
    "cv-40.txt",
    ",",
)
get_cased_word(gold_list)
gold_list, test_list = add_space(gold_list, test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
gold_rc = jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp = jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp = jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

1537
Cased words number is  1882
CP-WER
wer:  29.704853187281305
hits:  9700
substitutions:  2784
insertions:  459
deletions:  662
total words:  13146
P-WER
wer:  28.251939753537197
hits:  9893
substitutions:  2589
insertions:  461
deletions:  664
total words:  13146
C-WER
wer:  30.16720028459623
hits:  8280
substitutions:  2544
insertions:  428
deletions:  420
total words:  11244
wer:  28.352899324083957
hits:  8487
substitutions:  2334
insertions:  431
deletions:  423
total words:  11244


In [4]:
get_c_p_er_fr_wer_cv(
    wer=28.352899324083957,
    C_WER=30.16720028459623,
    P_WER=28.251939753537197,
)

Relative CER is  10.839532412327312
Relative PER is  27.655099894847528


In [15]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/books-transcription/test-clean-rich-book.txt","clean.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

2491
CP-WER
wer:  11.67195730282206
hits:  51019
substitutions:  3355
insertions:  377
deletions:  2960
total words:  57334
P-WER
wer:  9.636515854466808
hits:  52187
substitutions:  2186
insertions:  378
deletions:  2961
total words:  57334
C-WER
wer:  4.718562874251497
hits:  47860
substitutions:  2099
insertions:  124
deletions:  141
total words:  50100
wer:  2.283433133732535
hits:  49080
substitutions:  879
insertions:  124
deletions:  141
total words:  50100


In [9]:
get_c_p_er_fr_wer(wer=2.283433133732535,C_WER=4.718562874251497,P_WER=9.636515854466808,clean=True)

Relative CER is  31.685156576947435
Relative PER is  45.10547929216282


In [16]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/books-transcription/test-other-rich-book.txt","other.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
# gold_rp,test_rp= add_space(gold_rp,test_rp)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
print("WER")
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

2731
CP-WER
wer:  14.885284597914916
hits:  48065
substitutions:  4662
insertions:  468
deletions:  3194
total words:  55921
P-WER
wer:  12.719729618569053
hits:  49272
substitutions:  3459
insertions:  464
deletions:  3190
total words:  55921
C-WER
wer:  7.9617374811882815
hits:  44862
substitutions:  3334
insertions:  217
deletions:  311
total words:  48507
WER
wer:  5.384789824149092
hits:  46112
substitutions:  2084
insertions:  217
deletions:  311
total words:  48507


In [11]:
get_c_p_er_fr_wer(wer=5.384789824149092,C_WER=7.9617374811882815,P_WER=12.719729618569053,clean=False)

Relative CER is  29.53227085665711
Relative PER is  47.819336497257076


In [12]:
gold_list,test_list = get_gold_hyp_lists("/Users/ccui/Desktop/dictation/vasr/calculate_wer/ami_test_gold_process.txt","ihm.txt",',')
# get_cased_word(gold_list)
gold_list,test_list= add_space(gold_list,test_list)
print("CP-WER")
total_wer = get_detail_wer(gold_list, test_list)
# print("Total WER:", total_wer*100)
gold_rc= jiwer.ToLowerCase()(gold_list)
test_rc = jiwer.ToLowerCase()(test_list)
print("P-WER")
wer_rc = get_detail_wer(gold_rc, test_rc)
# print("WER removing case:", wer_rc*100)
gold_rp= jiwer.RemovePunctuation()(gold_list)
test_rp = jiwer.RemovePunctuation()(test_list)
print("C-WER")
wer_rp = get_detail_wer(gold_rp, test_rp)
# print("WER removing punctuation:", wer_rp*100)
gold_rc_rp= jiwer.RemovePunctuation()(gold_rc)
test_rc_rp = jiwer.RemovePunctuation()(test_rc)
wer_rc_rp = get_detail_wer(gold_rc_rp, test_rc_rp)

9474
CP-WER
wer:  45.432862351042026
hits:  63080
substitutions:  27363
insertions:  2911
deletions:  19823
total words:  110266
P-WER
wer:  43.21640396858506
hits:  65601
substitutions:  24765
insertions:  2988
deletions:  19900
total words:  110266
C-WER
wer:  42.06859136650852
hits:  56325
substitutions:  23922
insertions:  1866
deletions:  13759
total words:  94006
wer:  39.33897836308321
hits:  58927
substitutions:  21284
insertions:  1902
deletions:  13795
total words:  94006


In [13]:
get_c_p_er_fr_wer_ami(wer=39.33897836308321,C_WER=42.06859136650852,P_WER=43.21640396858506)

Relative CER is  33.648046157880934
Relative PER is  65.6334563345634
